# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import sys
sys.path.append("..")

In [2]:
import os
import glob

import psycopg2
import pandas as pd

from data_modelling import sql_queries
from data_modelling.connect import create_connection, create_engine

In [3]:
conn = create_connection("sparkifydb")
cur = conn.cursor()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("../data/song_data/")

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, lines=True).iloc[0]
df

artist_id                    ARNNKDK1187B98BBD5
artist_latitude                         45.8073
artist_location                  Zagreb Croatia
artist_longitude                        15.9676
artist_name                                Jinx
duration                                407.379
num_songs                                     1
song_id                      SOFNOQK12AB01840FC
title               Kutt Free (DJ Volume Remix)
year                                          0
Name: 0, dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
from data_modelling.etl.preparation import PreparerSongs, PreparerArtists, PreparerTime, PreparerUsers, PreparerSongplaysStaging

In [9]:
preparer_songs = PreparerSongs()

In [10]:
song_data = preparer_songs.transform(df)
song_data

[{'song_id': 'SOFNOQK12AB01840FC',
  'title': 'Kutt Free (DJ Volume Remix)',
  'artist_id': 'ARNNKDK1187B98BBD5',
  'year': 0,
  'duration': 407.37914}]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [38]:
from psycopg2.errors import UniqueViolation, NotNullViolation

In [48]:
def insert(query, records, cur, conn):
    for record in records:
        success = True
        try:
            cur.execute(query, record)
        except UniqueViolation:
            print("Record violates UNIQUE constraint.")
            success = False
        except NotNullViolation:
            print("Record violates NOT NULL constraint.")
            success = False
        except Exception as error:
            print("An unkown error has occured.")
            print(error)
            success = False
        if success:
            print("Record inserted successfully.")
        conn.commit()

In [52]:
insert(sql_queries.song_table_insert, song_data, cur, conn)

Record violates UNIQUE constraint.


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
preparer_artists = PreparerArtists()

In [15]:
artist_data = preparer_artists.transform(df)
artist_data

[{'artist_id': 'ARNNKDK1187B98BBD5',
  'name': 'Jinx',
  'location': 'Zagreb Croatia',
  'lattitude': 45.80726,
  'longitude': 15.967600000000001}]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [16]:
insert(sql_queries.artist_table_insert, artist_data, cur, conn)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [17]:
log_files = get_files("../data/log_data/")

In [18]:
filepath = log_files[0]

In [19]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Dominick,M,0,Norris,NaN,free,"Los Angeles-Long Beach-Anaheim, CA",GET,Home,1.540976e+12,44,None,200,1541635950796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",45
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
3,Collective Soul,Logged In,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Run (LP Version),200,1541639872796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
4,Taylor Swift,Logged In,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Love Story,200,1541640145796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [20]:
df = df[df["page"] == "NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
3,Collective Soul,Logged In,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Run (LP Version),200,1541639872796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
4,Taylor Swift,Logged In,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Love Story,200,1541640145796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
5,The Dixie Cups,Logged In,Aiden,M,4,Ramirez,120.05832,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Iko Iko,200,1541640378796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


In [21]:
preparer_time = PreparerTime()

In [22]:
time_data = preparer_time.transform(df)

In [23]:
time_data

[{'start_time': 1541639510796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541639702796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541639872796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640145796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640378796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640498796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640655796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640935796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541641099796,
  'hour': 1,
  'day': 8,


In [24]:
insert(sql_queries.time_table_insert, time_data, cur, conn)

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


In [26]:
preparer_users = PreparerUsers() 

In [27]:
users_data = preparer_users.transform(df)

In [28]:
insert(sql_queries.user_table_insert, users_data, cur, conn)

Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record already exists.
Record alre

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [29]:
df.head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
3,Collective Soul,Logged In,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Run (LP Version),200,1541639872796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
4,Taylor Swift,Logged In,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Love Story,200,1541640145796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
5,The Dixie Cups,Logged In,Aiden,M,4,Ramirez,120.05832,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Iko Iko,200,1541640378796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


In [30]:
preparer_songplays_staging = PreparerSongplaysStaging()

In [31]:
songplays_staging_data = preparer_songplays_staging.transform(df)

In [32]:
insert(sql_queries.songplay_staging_table_insert, songplays_staging_data, cur, conn)

In [33]:
query = """
SELECT songplays_staging.start_time,
       songplays_staging.user_id,
       songplays_staging.level,
       songs_artists.song_id,
       songs_artists.artist_id,
       songplays_staging.session_id,
       songplays_staging.location,
       songplays_staging.user_agent
  FROM songplays_staging
  LEFT JOIN (
       SELECT songs.song_id,
              artists.artist_id,
              songs.title,
              artists.name,
              songs.duration
         FROM songs
         LEFT JOIN artists
           ON songs.artist_id = artists.artist_id)
    AS songs_artists 
    ON songplays_staging.song_title = songs_artists.title
   AND songplays_staging.artist_name = songs_artists.name
   AND songplays_staging.song_duration = songs_artists.duration;
"""

In [34]:
songplays_data = pd.read_sql(query, conn).to_dict("records")

In [41]:
conn.commit()

In [42]:
insert(sql_queries.songplay_table_insert, songplays_data, cur, conn)

Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
Record violates NOT NULL constraint.
R

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.